In [8]:
import tensorflow as tf
import pathlib
import data
import time

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications        import InceptionV3
from tensorflow.keras.optimizers          import Adam
from tensorflow.keras.callbacks           import ModelCheckpoint, TensorBoard
from tensorflow.keras.layers              import Dense, GlobalAveragePooling2D
from tensorflow.keras.losses              import CategoricalCrossentropy
from tensorflow.keras                     import Sequential

In [9]:
def getImageDataGenerators(train_data_directory, validation_data_directory):
    train_image_generator      = ImageDataGenerator(rescale = 1./255)
    validation_image_generator = ImageDataGenerator(rescale = 1./255)
    
    #target_size - images will be re-sized to this size
    train_data_gen = train_image_generator.flow_from_directory(directory     = str(train_data_directory),
                                                               target_size   = (299, 299),
                                                               interpolation = "lanczos")
    
    validation_data_gen = validation_image_generator.flow_from_directory(directory    = str(validation_data_directory),
                                                                        target_size   = (299, 299),
                                                                        interpolation = "lanczos")
    return train_data_gen, validation_data_gen

In [10]:
def getModel(numClasses):
    base_model = InceptionV3(input_shape = (299, 299, 3),
                             include_top = False,
                             weights     = 'imagenet')
    #set to false because when trained, none of those weights will get updated
    base_model.trainable = False
    global_average_layer = GlobalAveragePooling2D()
    prediction_layer     = Dense(numClasses, activation = 'softmax')
    
    model = Sequential([base_model, global_average_layer, prediction_layer])
    
    model.compile(optimizer = Adam(lr = 0.0001),
                  loss      = CategoricalCrossentropy(from_logits = True),
                  metrics   = ['accuracy'])
    return model

In [11]:
#train weights of the base Sequential model
def fineTuneModel(model):
    
    #inceptionv3 layer
    base_model = model.layers[0]
    base_model.trainable = True
    
    #only layers after 300 will get weights updated (freeze all layers before fine_tune_at)
    # total # layers ~313, so when training again, it won't take up too much time
    #relistically, set to 2/3's? 
    for layer in base_model.layers[:300]:
        layer.trainable = False
        
    model.compile(optimizer = Adam(lr = 0.00001),
                  loss      = CategoricalCrossentropy(from_logits = True),
                  metrics   = ['accuracy'])
    return model

In [12]:
def trainModel(model, initial_epoch, epochs, train_data_gen, validation_data_gen, callbacks):
    history = model.fit(train_data_gen,
                        initial_epoch   = initial_epoch, 
                        epochs          = epochs, 
                        validation_data = validation_data_gen,
                        callbacks       = callbacks)
    return model, history

In [13]:
def main():
    dataObj    = data.Data()
    numClasses = dataObj.numClasses
    
    train_data_directory      = pathlib.Path(r"/home/jupyter/action-recognition/Frames/Train")
    validation_data_directory = pathlib.Path(r"/home/jupyter/action-recognition/Frames/Test")
    train_data_gen, validation_data_gen = getImageDataGenerators(train_data_directory, validation_data_directory)
    
    modelCheckpointDirectory = pathlib.Path(r"/home/jupyter/action-recognition/Callbacks/CNN/ModelCheckpoint")
    tensorboardDirectory     = pathlib.Path(r"/home/jupyter/action-recognition/Callbacks/CNN/Tensorboard")
    
    modelCheckpoint = ModelCheckpoint(filepath       = str(modelCheckpointDirectory/'CNN_{epoch:03d}_{val_loss:.2f}'),
                                      save_best_only = True)
    tensorboard = TensorBoard(log_dir = str(tensorboardDirectory/f'{int(time.time())}'))
    
    model = getModel(numClasses)
    trained_model, history = trainModel(model, 0, 1, train_data_gen, validation_data_gen, [])
    fine_tuned_model = fineTuneModel(trained_model)
    trained_model, history_fine = trainModel(fine_tuned_model, 1, 2, train_data_gen, validation_data_gen, [modelCheckpoint, tensorboard])

In [ ]:
#goal: get high accuracy
main()

Found 5660 images belonging to 3 classes.
Found 2291 images belonging to 3 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 177 steps, validate for 72 steps
  2/177 [..............................] - ETA: 20:36 - loss: 1.0634 - accuracy: 0.4375